In [3]:
# Import the dependencies
import pandas as pd
from pprint import pprint
from pymongo import MongoClient
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base

# NETFLIX

In [41]:
#raw data
CREDNET_df = pd.read_csv('RESOURCES2/NETFLIX/titles.csv')
CREDNET_df.head()

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945,TV-MA,51,['documentation'],['US'],1.0,NaN,NaN,NaN,0.600,NaN
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,"['drama', 'crime']",['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,"['drama', 'action', 'thriller', 'european']",['US'],NaN,tt0068473,7.7,107673.0,10.010,7.300
3,tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur, accompanied by his squire, recrui...",1975,PG,91,"['fantasy', 'action', 'comedy']",['GB'],NaN,tt0071853,8.2,534486.0,15.461,7.811
4,tm120801,The Dirty Dozen,MOVIE,12 American military prisoners in World War II...,1967,NaN,150,"['war', 'action']","['GB', 'US']",NaN,tt0061578,7.7,72662.0,20.398,7.600


In [18]:
#select desired attributes
NET_SELECTED = CREDNET_df[['id','title','type','release_year', 'genres','production_countries', 'imdb_score','imdb_votes']]
NET_SELECTED.head()

,id,title,type,release_year,genres,production_countries,imdb_score,imdb_votes
0,ts300399,Five Came Back: The Reference Films,SHOW,1945,['documentation'],['US'],NaN,NaN
1,tm84618,Taxi Driver,MOVIE,1976,"['drama', 'crime']",['US'],8.2,808582.0
2,tm154986,Deliverance,MOVIE,1972,"['drama', 'action', 'thriller', 'european']",['US'],7.7,107673.0
3,tm127384,Monty Python and the Holy Grail,MOVIE,1975,"['fantasy', 'action', 'comedy']",['GB'],8.2,534486.0
4,tm120801,The Dirty Dozen,MOVIE,1967,"['war', 'action']","['GB', 'US']",7.7,72662.0


# NETFLIX SHOWS CLEAN UP

In [20]:
#split the TVSHOWS 
NET_SHOWS = NET_SELECTED.loc[NET_SELECTED["type"] == "SHOW", :]
NET_SHOWS.head()

,id,title,type,release_year,genres,production_countries,imdb_score,imdb_votes
0,ts300399,Five Came Back: The Reference Films,SHOW,1945,['documentation'],['US'],NaN,NaN
5,ts22164,Monty Python's Flying Circus,SHOW,1969,"['comedy', 'european']",['GB'],8.8,73424.0
26,ts45948,Monty Python's Fliegender Zirkus,SHOW,1972,['comedy'],[],8.1,2151.0
35,ts20681,Seinfeld,SHOW,1989,['comedy'],['US'],8.9,308824.0
44,ts22082,Knight Rider,SHOW,1982,"['scifi', 'action', 'crime', 'drama']",['US'],6.9,34115.0


In [57]:
#drop NAN in all colums
CLEAN_NANS=NET_SHOWS.dropna() 
CLEAN_NANS.head()

,id,title,type,release_year,genres,production_countries,imdb_score,imdb_votes
5,ts22164,Monty Python's Flying Circus,SHOW,1969,"['comedy', 'european']",['GB'],8.8,73424.0
26,ts45948,Monty Python's Fliegender Zirkus,SHOW,1972,['comedy'],[],8.1,2151.0
35,ts20681,Seinfeld,SHOW,1989,['comedy'],['US'],8.9,308824.0
44,ts22082,Knight Rider,SHOW,1982,"['scifi', 'action', 'crime', 'drama']",['US'],6.9,34115.0
45,ts21715,Thomas & Friends,SHOW,1984,"['animation', 'family', 'comedy', 'fantasy', '...",['GB'],6.5,5104.0


In [56]:
#ascending order or IMDB SCORES AND VOTES
FINAL_NETSHOW = CLEAN_NANS.sort_values(['imdb_score', 'imdb_votes'],
              ascending = [False, False])
FINAL_NETSHOW.head()

,id,title,type,release_year,genres,production_countries,imdb_score,imdb_votes
2658,ts265844,#ABtalks,SHOW,2018,[],[],9.6,7.0
216,ts4,Breaking Bad,SHOW,2008,"['drama', 'crime', 'thriller']",['US'],9.5,1775990.0
564,ts160526,Khawatir,SHOW,2005,['reality'],[],9.5,3115.0
233,ts3371,Avatar: The Last Airbender,SHOW,2005,"['scifi', 'animation', 'action', 'family', 'fa...",['US'],9.3,303666.0
3147,ts85398,Our Planet,SHOW,2019,['documentation'],['GB'],9.3,42259.0


# NETFLIX MOVIES CLEAN UP

In [19]:
#split the MOVIES
NET_MOVIES = NET_SELECTED.loc[NET_SELECTED["type"] == "MOVIE", :]
NET_MOVIES.head()

,id,title,type,release_year,genres,production_countries,imdb_score,imdb_votes
1,tm84618,Taxi Driver,MOVIE,1976,"['drama', 'crime']",['US'],8.2,808582.0
2,tm154986,Deliverance,MOVIE,1972,"['drama', 'action', 'thriller', 'european']",['US'],7.7,107673.0
3,tm127384,Monty Python and the Holy Grail,MOVIE,1975,"['fantasy', 'action', 'comedy']",['GB'],8.2,534486.0
4,tm120801,The Dirty Dozen,MOVIE,1967,"['war', 'action']","['GB', 'US']",7.7,72662.0
6,tm70993,Life of Brian,MOVIE,1979,['comedy'],['GB'],8.0,395024.0


In [54]:
#drop NAN in all colums
CLEAN_NANM=NET_MOVIES.dropna() 
CLEAN_NANM.head()

,id,title,type,release_year,genres,production_countries,imdb_score,imdb_votes
1,tm84618,Taxi Driver,MOVIE,1976,"['drama', 'crime']",['US'],8.2,808582.0
2,tm154986,Deliverance,MOVIE,1972,"['drama', 'action', 'thriller', 'european']",['US'],7.7,107673.0
3,tm127384,Monty Python and the Holy Grail,MOVIE,1975,"['fantasy', 'action', 'comedy']",['GB'],8.2,534486.0
4,tm120801,The Dirty Dozen,MOVIE,1967,"['war', 'action']","['GB', 'US']",7.7,72662.0
6,tm70993,Life of Brian,MOVIE,1979,['comedy'],['GB'],8.0,395024.0


In [55]:
#ascending order or IMDB SCORES AND VOTES
FINAL_NETMOVIE = CLEAN_NANM.sort_values(['imdb_score', 'imdb_votes'],
              ascending = [False, False])
FINAL_NETMOVIE.head()

,id,title,type,release_year,genres,production_countries,imdb_score,imdb_votes
4771,tm996762,Major,MOVIE,2022,"['action', 'drama']",['IN'],9.1,7576.0
1092,tm250247,Chhota Bheem & Krishna vs Zimbara,MOVIE,2013,['animation'],[],9.1,16.0
3271,tm853783,David Attenborough: A Life on Our Planet,MOVIE,2020,['documentation'],['GB'],8.9,31625.0
2547,tm432327,C/o Kancharapalem,MOVIE,2018,['drama'],['IN'],8.9,6842.0
237,tm92641,Inception,MOVIE,2010,"['action', 'scifi', 'music', 'thriller']","['US', 'GB']",8.8,2294231.0


# PARAMOUNT

In [42]:
CREDPAR_df = pd.read_csv('RESOURCES2/PARAMOUNT/titles.csv')
CREDPAR_df.head()

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,tm19248,The General,MOVIE,"During America’s Civil War, Union spies steal ...",1926,NaN,78,"['action', 'drama', 'war', 'western', 'comedy'...",['US'],NaN,tt0017925,8.2,89766.0,8.647,8.0
1,tm83884,His Girl Friday,MOVIE,"Hildy, the journalist former wife of newspaper...",1940,NaN,92,"['comedy', 'drama', 'romance']",['US'],NaN,tt0032599,7.8,57835.0,11.270,7.4
2,tm19424,Detour,MOVIE,"The life of Al Roberts, a pianist in a New Yor...",1945,NaN,66,"['thriller', 'drama', 'crime']",['US'],NaN,tt0037638,7.3,17233.0,7.757,7.2
3,tm112005,Marihuana,MOVIE,A young girl named Burma attends a beach party...,1936,NaN,57,"['crime', 'drama']",['US'],NaN,tt0026683,4.0,864.0,3.748,3.6
4,tm22806,Intolerance: Love's Struggle Throughout the Ages,MOVIE,"The story of a poor young woman, separated by ...",1916,NaN,197,"['history', 'drama']",['US'],NaN,tt0006864,7.7,15242.0,9.412,7.2


In [21]:
#select desired attributes
PAR_SELECTED = CREDPAR_df[['id','title','type','release_year', 'genres','production_countries', 'imdb_score','imdb_votes']]
PAR_SELECTED.head()

,id,title,type,release_year,genres,production_countries,imdb_score,imdb_votes
0,tm19248,The General,MOVIE,1926,"['action', 'drama', 'war', 'western', 'comedy'...",['US'],8.2,89766.0
1,tm83884,His Girl Friday,MOVIE,1940,"['comedy', 'drama', 'romance']",['US'],7.8,57835.0
2,tm19424,Detour,MOVIE,1945,"['thriller', 'drama', 'crime']",['US'],7.3,17233.0
3,tm112005,Marihuana,MOVIE,1936,"['crime', 'drama']",['US'],4.0,864.0
4,tm22806,Intolerance: Love's Struggle Throughout the Ages,MOVIE,1916,"['history', 'drama']",['US'],7.7,15242.0


# PARAMOUNT SHOW CLEAN UP

In [22]:
#split the TVSHOWS 
PAR_SHOWS = PAR_SELECTED.loc[PAR_SELECTED["type"] == "SHOW", :]
PAR_SHOWS.head()

,id,title,type,release_year,genres,production_countries,imdb_score,imdb_votes
550,ts22109,The Twilight Zone,SHOW,1959,"['scifi', 'drama', 'fantasy', 'horror', 'thril...",['US'],9.1,80605.0
551,ts21236,Star Trek,SHOW,1966,"['scifi', 'action']",['US'],8.4,81281.0
555,ts20005,I Love Lucy,SHOW,1951,"['comedy', 'family']",['US'],8.5,25944.0
557,ts21682,Happy Days,SHOW,1974,"['comedy', 'history', 'music', 'family', 'thri...",['US'],7.3,32.0
559,ts22170,Gunsmoke,SHOW,1955,"['action', 'western']",['US'],8.0,6859.0


In [59]:
#drop NAN in all colums
CLEAN_NANpS=PAR_SHOWS.dropna() 
CLEAN_NANpS.head()

,id,title,type,release_year,genres,production_countries,imdb_score,imdb_votes
550,ts22109,The Twilight Zone,SHOW,1959,"['scifi', 'drama', 'fantasy', 'horror', 'thril...",['US'],9.1,80605.0
551,ts21236,Star Trek,SHOW,1966,"['scifi', 'action']",['US'],8.4,81281.0
555,ts20005,I Love Lucy,SHOW,1951,"['comedy', 'family']",['US'],8.5,25944.0
557,ts21682,Happy Days,SHOW,1974,"['comedy', 'history', 'music', 'family', 'thri...",['US'],7.3,32.0
559,ts22170,Gunsmoke,SHOW,1955,"['action', 'western']",['US'],8.0,6859.0


In [61]:
#ascending order or IMDB SCORES AND VOTES
FINAL_PARSHOWS = CLEAN_NANpS.sort_values(['imdb_score', 'imdb_votes'],
              ascending = [False, False])
FINAL_PARSHOWS.head()

,id,title,type,release_year,genres,production_countries,imdb_score,imdb_votes
2205,ts107506,Orangutan Jungle School,SHOW,2018,['documentation'],"['AU', 'US']",9.4,143.0
1249,ts3371,Avatar: The Last Airbender,SHOW,2005,"['scifi', 'family', 'fantasy', 'animation', 'a...",['US'],9.3,297336.0
550,ts22109,The Twilight Zone,SHOW,1959,"['scifi', 'drama', 'fantasy', 'horror', 'thril...",['US'],9.1,80605.0
1741,ts166066,L.A. Frock Stars,SHOW,2013,['reality'],['ZA'],9.0,41.0
2080,ts125631,Ol Pejeta Diaries,SHOW,2015,[],[],9.0,6.0


# PARAMOUNT MOVIES CLEAN UP

In [23]:
#split the MOVIES
PAR_MOVIES = PAR_SELECTED.loc[PAR_SELECTED["type"] == "MOVIE", :]
PAR_MOVIES.head()

,id,title,type,release_year,genres,production_countries,imdb_score,imdb_votes
0,tm19248,The General,MOVIE,1926,"['action', 'drama', 'war', 'western', 'comedy'...",['US'],8.2,89766.0
1,tm83884,His Girl Friday,MOVIE,1940,"['comedy', 'drama', 'romance']",['US'],7.8,57835.0
2,tm19424,Detour,MOVIE,1945,"['thriller', 'drama', 'crime']",['US'],7.3,17233.0
3,tm112005,Marihuana,MOVIE,1936,"['crime', 'drama']",['US'],4.0,864.0
4,tm22806,Intolerance: Love's Struggle Throughout the Ages,MOVIE,1916,"['history', 'drama']",['US'],7.7,15242.0


In [58]:
#drop NAN in all colums
CLEAN_NANpM=PAR_MOVIES.dropna() 
CLEAN_NANpM.head()

,id,title,type,release_year,genres,production_countries,imdb_score,imdb_votes
0,tm19248,The General,MOVIE,1926,"['action', 'drama', 'war', 'western', 'comedy'...",['US'],8.2,89766.0
1,tm83884,His Girl Friday,MOVIE,1940,"['comedy', 'drama', 'romance']",['US'],7.8,57835.0
2,tm19424,Detour,MOVIE,1945,"['thriller', 'drama', 'crime']",['US'],7.3,17233.0
3,tm112005,Marihuana,MOVIE,1936,"['crime', 'drama']",['US'],4.0,864.0
4,tm22806,Intolerance: Love's Struggle Throughout the Ages,MOVIE,1916,"['history', 'drama']",['US'],7.7,15242.0


In [64]:
#ascending order or IMDB SCORES AND VOTES
FINAL_PARMOVIE = CLEAN_NANpM.sort_values(['imdb_score', 'imdb_votes'],
              ascending = [False, False])
FINAL_PARMOVIE.head()

,id,title,type,release_year,genres,production_countries,imdb_score,imdb_votes
2775,tm1175333,The Loyola Project,MOVIE,2022,['documentation'],[],9.4,10.0
549,tm155580,The Godfather,MOVIE,1972,"['drama', 'crime']",['US'],9.2,1780564.0
552,tm112061,The Godfather: Part II,MOVIE,1974,"['drama', 'crime']",['US'],9.0,1230117.0
1073,tm112130,Pulp Fiction,MOVIE,1994,"['thriller', 'crime', 'drama']",['US'],8.9,1982722.0
1837,tm139219,Florence Foster Jenkins,MOVIE,2016,"['drama', 'crime', 'music', 'comedy']","['US', 'GB']",8.9,9.0


In [10]:
mergedDF = titlesDF.merge(creditsDF, how='inner', on='id')
mergedDF = mergedDF.rename(columns={'id':'MovieID', 'title':'Title', 'type':'Type',
                                   'description':'Description', 'release_year':'ReleaseYear',
                                   'age_certification': 'AgeCertification', 'runtime':'Runtime',
                                   'genres':'Genres', 'production_countries':'ProductionCountries',
                                   'seasons':'Seasons', 'imdb_id':'imdbID', 'imdb_score':'imdbScore',
                                   'imdb_votes':'imdbVotes', 'tmdb_popularity':'tmdbPopularity',
                                   'tmdb_score':'tmdbScore', 'person_id':'PersonID',
                                   'name':'Name', 'character':'Character', 'role':'Role'})
mergedDF

,MovieID,Title,Type,Description,ReleaseYear,AgeCertification,Runtime,Genres,ProductionCountries,Seasons,imdbID,imdbScore,imdbVotes,tmdbPopularity,tmdbScore,PersonID,Name,Character,Role
0,tm19248,The General,MOVIE,"During America’s Civil War, Union spies steal ...",1926,NaN,78,"['action', 'drama', 'war', 'western', 'comedy'...",['US'],NaN,tt0017925,8.2,89766.0,8.647,8.0,21174,Buster Keaton,Johnny Gray,ACTOR
1,tm19248,The General,MOVIE,"During America’s Civil War, Union spies steal ...",1926,NaN,78,"['action', 'drama', 'war', 'western', 'comedy'...",['US'],NaN,tt0017925,8.2,89766.0,8.647,8.0,28713,Marion Mack,Annabelle Lee,ACTOR
2,tm19248,The General,MOVIE,"During America’s Civil War, Union spies steal ...",1926,NaN,78,"['action', 'drama', 'war', 'western', 'comedy'...",['US'],NaN,tt0017925,8.2,89766.0,8.647,8.0,28714,Glen Cavender,Captain Anderson,ACTOR
3,tm19248,The General,MOVIE,"During America’s Civil War, Union spies steal ...",1926,NaN,78,"['action', 'drama', 'war', 'western', 'comedy'...",['US'],NaN,tt0017925,8.2,89766.0,8.647,8.0,28715,Jim Farley,General Thatcher,ACTOR
4,tm19248,The General,MOVIE,"During America’s Civil War, Union spies steal ...",1926,NaN,78,"['action', 'drama', 'war', 'western', 'comedy'...",['US'],NaN,tt0017925,8.2,89766.0,8.647,8.0,27348,Frederick Vroom,A Southern General,ACTOR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39837,tm1045408,R.I.P. T-Shirts,MOVIE,"A small custom T-shirt shop in Washington, DC,...",2021,NaN,30,['documentation'],['US'],NaN,NaN,NaN,NaN,1.673,NaN,357110,Kate Davis,NaN,DIRECTOR
39838,tm1045408,R.I.P. T-Shirts,MOVIE,"A small custom T-shirt shop in Washington, DC,...",2021,NaN,30,['documentation'],['US'],NaN,NaN,NaN,NaN,1.673,NaN,68620,David Heilbroner,NaN,DIRECTOR
39839,tm1106415,Destination Porto: The Unimaginable Journey,MOVIE,Follow soccer journalist Guillem Belagué as he...,2021,PG,85,['documentation'],[],NaN,tt15430722,7.7,8.0,0.600,NaN,2078684,Guillem Balagué,NaN,ACTOR
39840,tm1106415,Destination Porto: The Unimaginable Journey,MOVIE,Follow soccer journalist Guillem Belagué as he...,2021,PG,85,['documentation'],[],NaN,tt15430722,7.7,8.0,0.600,NaN,2078564,Alex Gale,NaN,DIRECTOR


In [6]:
mergedDF.dtypes

id                       object
title                    object
type                     object
description              object
release_year              int64
age_certification        object
runtime                   int64
genres                   object
production_countries     object
seasons                 float64
imdb_id                  object
imdb_score              float64
imdb_votes              float64
tmdb_popularity         float64
tmdb_score              float64
person_id                 int64
name                     object
character                object
role                     object
dtype: object